# Uso di Pipeline di Machine Learning in Spark

Parteciperemo "virtualmente" alla competizione sul Titanic data set organizzata sul portale [Kaggle](https://www.kaggle.com/c/titanic/overview) da cui trarremo i due data set di addestramento e test. Nel resto dell'esempio i dati si troveranno all'interno del file system Hadoop.

Il training set consta di 891 righe e il test set di 418 e non ha la colonna dei sopravvissuti. Addestreremo un classificatore Random Forests su una griglia di iperparametri valutati con una procedura di 10-fold crossvalidation.

Dapprima ci preoccuperemo di gestire le feature mancanti e, successivamente, costruiremo la Pipeline Spark di addestramento e valutazione.

In [1]:
# inizializziamo la SparkSession e importiamo le librerie
import findspark

location = findspark.find()
findspark.init(location)

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import IndexToString
from pyspark.ml.classification import RandomForestClassifier 
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

spark = SparkSession \
    .builder \
    .appName("Spark ML example on titanic data ") \
    .getOrCreate()

23/05/10 15:39:45 WARN Utils: Your hostname, deeplearning resolves to a loopback address: 127.0.1.1; using 147.163.26.113 instead (on interface enp6s0)
23/05/10 15:39:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/10 15:39:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/10 15:39:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Carichiamo  i data set e mostriamo il loro schema

In [70]:

trainDF = spark \
    .read \
    .csv('/home/rpirrone/data/train.csv',header = 'True', inferSchema='True')

testDF = spark \
    .read \
    .csv('/home/rpirrone/data/test.csv',header = 'True', inferSchema='True')


In [71]:
print("training set")
trainDF.printSchema()

print("\n\ntest set")
testDF.printSchema()

training set
root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



test set
root
 |-- PassengerId: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



Per osservare il nostro data set, inziamo contando imbarcati e sopravvisuti ed estraendo le statistiche di base delle feature numeriche.

In [72]:
passengers_count = trainDF.count()
survived_count = trainDF.filter("Survived == 1").count()

print(f"Passeggeri: {passengers_count}\nDi cui sopravvissuti: {survived_count}")

trainDF.describe()['summary','Pclass','Age','SibSp','Parch','Fare'].show()

print(f"Passeggeri: {testDF.count()}")
testDF.describe()['summary','Pclass','Age','SibSp','Parch','Fare'].show()


Passeggeri: 891
Di cui sopravvissuti: 342
+-------+------------------+------------------+------------------+-------------------+-----------------+
|summary|            Pclass|               Age|             SibSp|              Parch|             Fare|
+-------+------------------+------------------+------------------+-------------------+-----------------+
|  count|               891|               714|               891|                891|              891|
|   mean| 2.308641975308642| 29.69911764705882|0.5230078563411896|0.38159371492704824| 32.2042079685746|
| stddev|0.8360712409770491|14.526497332334035|1.1027434322934315| 0.8060572211299488|49.69342859718089|
|    min|                 1|              0.42|                 0|                  0|              0.0|
|    max|                 3|              80.0|                 8|                  6|         512.3292|
+-------+------------------+------------------+------------------+-------------------+-----------------+

Passeggeri: 

Contiamo i valori nulli nelle diverse colonne tramite una UDF

In [73]:
def null_value_count(df):
  null_columns_counts = []
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)

null_columns_count_list_train = null_value_count(trainDF)
null_columns_count_list_test = null_value_count(testDF)

spark.createDataFrame(null_columns_count_list_train, ['Column_With_Null_Value', 'Null_Values_Count']).show()

spark.createDataFrame(null_columns_count_list_test, ['Column_With_Null_Value', 'Null_Values_Count']).show()

+----------------------+-----------------+
|Column_With_Null_Value|Null_Values_Count|
+----------------------+-----------------+
|                   Age|              177|
|                 Cabin|              687|
|              Embarked|                2|
+----------------------+-----------------+

+----------------------+-----------------+
|Column_With_Null_Value|Null_Values_Count|
+----------------------+-----------------+
|                   Age|               86|
|                  Fare|                1|
|                 Cabin|              327|
+----------------------+-----------------+



### Gestione dei valori nulli

- Age: calcoleremo l'età media dei paseggeri raggruppati per 'titolo' nel nome (Mr, Mrs, Miss, ...) poiché questo corrisponde a delle fasce di età piuttosto precise
- Fare: faremo il drop perché correlata con 'Pclass'
- Cabin: faremo il drop della feature perché non interessa
- Embarked: non è molto rilevante ai fini della classificazione, ma faremo imputazione con il valore più frequente e cioé 'S'


In [74]:
# estraiamo la lista dei titoli su entrambi i dataframe e aggiungiamo una nuova colonna con l'iniziale

trainDF=trainDF.withColumn("Initial",regexp_extract(col("Name"),".*, (.*?)\\..*",1))
testDF=testDF.withColumn("Initial",regexp_extract(col("Name"),".*, (.*?)\\..*",1))

trainDF.select("Initial").distinct().show()
testDF.select("Initial").distinct().show()


+------------+
|     Initial|
+------------+
|         Don|
|        Miss|
|         Col|
|         Rev|
|        Lady|
|      Master|
|         Mme|
|        Capt|
|          Mr|
|          Dr|
|         Mrs|
|         Sir|
|    Jonkheer|
|        Mlle|
|       Major|
|          Ms|
|the Countess|
+------------+

+-------+
|Initial|
+-------+
|   Dona|
|   Miss|
|    Col|
|    Rev|
| Master|
|     Mr|
|     Dr|
|    Mrs|
|     Ms|
+-------+



In [75]:
dataset = trainDF.unionByName(testDF,allowMissingColumns=True)

dataset.tail(10)

[Row(PassengerId=1300, Survived=None, Pclass=3, Name='Riordan, Miss. Johanna Hannah""""', Sex='female', Age=None, SibSp=0, Parch=0, Ticket='334915', Fare=7.7208, Cabin=None, Embarked='Q', Initial='Miss'),
 Row(PassengerId=1301, Survived=None, Pclass=3, Name='Peacock, Miss. Treasteall', Sex='female', Age=3.0, SibSp=1, Parch=1, Ticket='SOTON/O.Q. 3101315', Fare=13.775, Cabin=None, Embarked='S', Initial='Miss'),
 Row(PassengerId=1302, Survived=None, Pclass=3, Name='Naughton, Miss. Hannah', Sex='female', Age=None, SibSp=0, Parch=0, Ticket='365237', Fare=7.75, Cabin=None, Embarked='Q', Initial='Miss'),
 Row(PassengerId=1303, Survived=None, Pclass=1, Name='Minahan, Mrs. William Edward (Lillian E Thorpe)', Sex='female', Age=37.0, SibSp=1, Parch=0, Ticket='19928', Fare=90.0, Cabin='C78', Embarked='Q', Initial='Mrs'),
 Row(PassengerId=1304, Survived=None, Pclass=3, Name='Henriksson, Miss. Jenny Lovisa', Sex='female', Age=28.0, SibSp=0, Parch=0, Ticket='347086', Fare=7.775, Cabin=None, Embarked=

Calcoliamo le statistiche su 'Age'

In [76]:
dataset.groupBy('Initial').agg(expr('AVG(Age) AS Average'),min('Age'),max('Age')).show()


+------------+------------------+--------+--------+
|     Initial|           Average|min(Age)|max(Age)|
+------------+------------------+--------+--------+
|         Don|              40.0|    40.0|    40.0|
|        Miss|21.774238095238097|    0.17|    63.0|
|         Col|              54.0|    47.0|    60.0|
|         Rev|             41.25|    27.0|    57.0|
|        Lady|              48.0|    48.0|    48.0|
|      Master| 5.482641509433963|    0.33|    14.5|
|         Mme|              24.0|    24.0|    24.0|
|        Capt|              70.0|    70.0|    70.0|
|          Mr| 32.25215146299484|    11.0|    80.0|
|          Dr| 43.57142857142857|    23.0|    54.0|
|         Mrs| 36.99411764705882|    14.0|    76.0|
|         Sir|              49.0|    49.0|    49.0|
|    Jonkheer|              38.0|    38.0|    38.0|
|        Mlle|              24.0|    24.0|    24.0|
|       Major|              48.5|    45.0|    52.0|
|          Ms|              28.0|    28.0|    28.0|
|the Countes

In [77]:
avg_age = dataset.groupBy('Initial').agg(expr('AVG(Age) as Average'))
avg_age = avg_age.withColumnRenamed('Initial','Init')

Eseguiamo l'imputazione di 'Age' secondo il valor medio dell'età corrispondente al titolo del nome.

In [78]:
trainDF = trainDF.join(avg_age,trainDF.Initial == avg_age.Init)
trainDF.drop('Init')

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string, Initial: string, Average: double]

In [80]:
testDF = testDF.join(avg_age,testDF.Initial == avg_age.Init).drop('Init')

In [81]:
trainDF = trainDF.withColumn('Age',\
    when(trainDF.Age.isNull(),trainDF.Average))

testDF = testDF.withColumn('Age',\
    when(testDF.Age.isNull(),testDF.Average))

trainDF.drop('Average')
testDF.drop('Average')


DataFrame[PassengerId: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string, Initial: string]

In [83]:
# Calcoliamo il luogo di imbarco più frequente e imputiamo al valore mancante
dataset.groupBy('Embarked').count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|  123|
|    null|    2|
|       C|  270|
|       S|  914|
+--------+-----+



In [82]:
trainDF = trainDF.withColumn('Embarked',\
    when(trainDF.Embarked.isNull(),'S'))


Creiamo una colonna "FamilySize" che somma "Parch" (Parents/children) e "Sibsp" (Sibling/spouses) + 1

In [84]:
trainDF = trainDF.withColumn("FamilySize",col('SibSp')+col('Parch')+1)
testDF = testDF.withColumn("FamilySize",col('SibSp')+col('Parch')+1)

trainDF.groupBy("FamilySize").count().show()
testDF.groupBy("FamilySize").count().show()

+----------+-----+
|FamilySize|count|
+----------+-----+
|         1|  537|
|         6|   22|
|         3|  102|
|         5|   15|
|         4|   29|
|         8|    6|
|         7|   12|
|        11|    7|
|         2|  161|
+----------+-----+

+----------+-----+
|FamilySize|count|
+----------+-----+
|         1|  253|
|         6|    3|
|         3|   57|
|         5|    7|
|         4|   14|
|         8|    2|
|         7|    4|
|        11|    4|
|         2|   74|
+----------+-----+



Vista la grande prevalenza di passeggeri che viaggiano da soli, Creiamo anche una colonna binaria apposita "Alone" per indicar coloro che hanno "FamilySize = 1"

In [85]:
trainDF = trainDF.withColumn('Alone',lit(0))
trainDF = trainDF.withColumn("Alone",when(trainDF["FamilySize"] == 1, 1).otherwise(trainDF["Alone"]))

testDF = testDF.withColumn('Alone',lit(0))
testDF = testDF.withColumn("Alone",when(testDF["FamilySize"] == 1, 1).otherwise(testDF["Alone"]))


### Pipeline di addestramento del classificatore

Come già sappiamo, una pipeline è una sequenza ordinata di:

- Transformers: algoritmi che trasformano effettivamente un dataframe (metodo `transform()`)
- Estimators: algoritmi che si addestrano sui dati per generare un Transformer (metodo `fit()` che usa la "eager execution" ovvero l'esecuzione immediata)
- Evaluators: algoritmi di calcolo dei criteri di valutazione delle performance

Sono Transformer anche gli algoritmi di gestione delle feature in ingresso e uscita. La pipeline può contenere anche algoritmi per il tuning degli iperparametri.

Eseguiremo il `fit` degli Estimators (incluso quindi l'addestramento vero e proprio) sul trainDF. mentre il `transform` del modello addestrato (che è un Transformer) e la valutazione sul testDF.

Indicizziamo i dati categorici (tranne "Survived") su un unico dataframe contenente training e test set. Questa soluzione si giustifica con il fatto che lo StringIndexer crea degli indici numerici con le frequenze di occorrenza delle etichette catgoriche, quindi è più opportuno farne il "fit" su tutti i dati.

In [86]:
labelDF = trainDF["Pclass","Sex","Embarked","Initial","Alone"].\
          unionByName(testDF["Pclass","Sex","Embarked","Initial","Alone"])

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(labelDF) for column in ["Pclass","Sex","Embarked","Initial","Alone"]]

# Trasformiamo il data set per ottenere realmente le colonne indicizzate e utilizzarle poi come feature
for indexer in indexers:
    trainDF=indexer.transform(trainDF)
    testDF=indexer.transform(testDF)

labelIndexer = StringIndexer(inputCol='Survived',outputCol='Survived_index').fit(trainDF)



Creiamo un mapping per le label predette dall'algoritmo che, dopo l'addestramento, restituirà una colonna "prediction" indicizzata.

In [87]:
labelConverter = IndexToString(inputCol='prediction',outputCol='predictedLabel').setLabels(labelIndexer.labels)


Assembliamo le feature con un VectorAssembler cioè un Transformer che crea il vettore di tutte le feature concatenate. Il classificatore richiederà il vettore delle feature in un'unica colonna.

Escludiamo le feature per le quali avevamo deciso di fare il drop.

In [88]:
assembler = VectorAssembler()\
        .setInputCols(["Age", "SibSp", "Parch", "FamilySize", "Pclass_index","Sex_index","Embarked_index","Initial_index","Alone_index"])\
        .setOutputCol("Features")\
        .setHandleInvalid("keep")

Creiamo il classificatore come Evaluator impostando le colonne delle feature e delle label Il "fit" di questo Evaluator restituisce un Transformer di tipo RandomForestClassificationModel il cui metodo "transform" possiamo usare per predire i sopravvissuti sul testDF.

In [89]:
randomForest = RandomForestClassifier().setFeaturesCol("Features").setLabelCol("Survived_index")

Creiamo la pipeline per una singola classificazione. La pipeline, nel suo complesso, è un Evaluator il cui metodo "fit" genererà un Transformer per il test set.

In [90]:
pipeline = Pipeline().setStages([\
                                labelIndexer,\
                                assembler,\
                                randomForest,\
                                labelConverter])

Costruiamo l'algoritmo di addestramento, come una 10-fold Cross-validation che si addestra su una griglia di iperparametri:

- l'indice di Gini, e l'entropia per controllare la purezza dei nodi foglia
- il numero di bin cioè di categorie da generare per ogni feature categorica
- la profondità massima dell'albero

In [91]:
paramGrid = ParamGridBuilder().addGrid(randomForest.maxBins,[25, 28, 31])\
                              .addGrid(randomForest.maxDepth,[4,6,8])\
                              .addGrid(randomForest.impurity,["entropy","gini"])\
                              .build()

Generiamo l'evaluator che inseriremo nell'algoritmo di addestramento. Questo utilizzerà la metrica Area Under Precision-Recall Curve (AUPRC) che si adatta meglio ad una classificazione binaria con classi sbilanciate, com'è il nostro caso in cui i sopravvissuti sono pochi.

La AUPRC va esplicitamente confrontata con una baseline di riferimento definita come $\frac{P}{P+N}$ che ha valori diversi se la classe positiva contiene pochi campioni. 

In [92]:
evaluator = BinaryClassificationEvaluator().setLabelCol("Survived_index")\
                                           .setMetricName("areaUnderPR")
                                           

Infine costruiamo l'Estimator che implementa la 10-fold cross-validation, inserendo la pipeline come Estimator dei dati, l'evaluator e la griglia degli iperparametri per l'addestramento.

In [93]:
cv = CrossValidator().setEstimator(pipeline)\
                     .setEvaluator(evaluator)\
                     .setEstimatorParamMaps(paramGrid)\
                     .setNumFolds(10)

In [94]:
# Addesrtiamo sul training set
cvModel = cv.fit(trainDF)

23/05/10 17:53:28 WARN BlockManager: Putting block rdd_1514_0 failed due to exception org.apache.spark.SparkException: Failed to execute user defined function (StringIndexerModel$$Lambda$4257/342310508: (string) => double).
23/05/10 17:53:28 WARN BlockManager: Block rdd_1514_0 could not be removed as it was not found on disk or in memory
23/05/10 17:53:28 ERROR Executor: Exception in task 0.0 in stage 574.0 (TID 656)
org.apache.spark.SparkException: Failed to execute user defined function (StringIndexerModel$$Lambda$4257/342310508: (string) => double)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage5.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIt

Py4JJavaError: An error occurred while calling o1896.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 574.0 failed 1 times, most recent failure: Lost task 0.0 in stage 574.0 (TID 656) (147.163.26.113 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (StringIndexerModel$$Lambda$4257/342310508: (string) => double)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage5.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at org.apache.spark.sql.execution.colum

23/05/10 17:53:29 WARN BlockManager: Putting block rdd_1514_0 failed due to exception org.apache.spark.SparkException: Failed to execute user defined function (StringIndexerModel$$Lambda$4257/342310508: (string) => double).
23/05/10 17:53:29 WARN BlockManager: Block rdd_1514_0 could not be removed as it was not found on disk or in memory
23/05/10 17:53:29 ERROR Executor: Exception in task 0.0 in stage 575.0 (TID 657)
org.apache.spark.SparkException: Failed to execute user defined function (StringIndexerModel$$Lambda$4257/342310508: (string) => double)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage5.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIt

### Predizione e valutazione dei risultati

Facciamo la predizione sul test set e analizziamo l'accuratezza sul training set. Confronteremo la misura AUPRC con la baseline per la classe dei sopravvisuti.

In [27]:
predictions = cvModel.transform(testDF)

23/05/10 14:03:15 WARN StringIndexerModel: Input column Survived does not exist during transformation. Skip StringIndexerModel for this column.


In [31]:
performance = cvModel.transform(trainDF)
auprc = evaluator.evaluate(performance)
print(f"Area Under PR Curve: {(100*auprc):05.2f}%\nBaseline: {100*survived_count/passengers_count:05.2f}%")

Area Under PR Curve: 90.31%
Baseline: 38.38%


Aggiungiamo anche la misura di AUC richiedendola al nostro `evaluator`

In [35]:
auroc = evaluator.evaluate(performance,{evaluator.metricName: 'areaUnderROC'})
print(f"Area Under ROC: {(100*auroc):05.2f}%")

Area Under ROC: 91.42%


In [32]:
# Creiamo il file dei risultati per sottometterlo sul sito della competizione

# Salviamo il modello addestrato

cvModel.write().overwrite().save('/home/rpirrone/data/RF_10xfold_cv_model')

In [33]:
# Salviamo in csv con le colonne "PassengerId" "Survived"

predictions\
  .withColumn("Survived", col("predictedLabel"))\
  .select("PassengerId", "Survived")\
  .coalesce(1)\
  .write\
  .csv('/home/rpirrone/data/titanic_predictions.csv',\
    header=True, mode='overwrite')
